# Import required packages

In [ ]:
!pip install transformers datasets torch scikit-learn
#!pip install numpy==1.23.4
!pip install numpy==1.26.4

**BertForSequenceClassification** is a model provided by the Hugging Face Transformers library. It is a fine-tuned variant of BERT (Bidirectional Encoder Representations from Transformers) designed specifically for sequence classification tasks.

# Read the data

In [ ]:
import pandas as pd
data=pd.read_excel('/content/IMDB Dataset_sample.xlsx')
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


# Preprocess the data we need to do

- here generally we need to do data preprocessing

- that is your task

# Train test split

In [ ]:
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Convert sentiment to numerical labels
data['sentiment'] = data['sentiment'].map({"positive": 1, "negative": 0})

# Split the data into train and test sets
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)

# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
type(train_df)

pandas.core.frame.DataFrame

In [ ]:
train_dataset

Dataset({
    features: ['review', 'sentiment', '__index_level_0__'],
    num_rows: 891
})

In [ ]:
train_dataset[:5]

{'review': ['This film was pretty good. I am not too big a fan of baseball, but this is a movie that was made to help understand the meaning of love, determination, heart, etc.<br /><br />Danny Glover, Joseph Gordon-Levitt, Brenda Fricker, Christopher Lloyd, Tony Danza, and Milton Davis Jr. are brought in with a variety of talented actors and understanding of the sport. The plot was believable, and I love the message. William Dear and the guys put together a great movie.<br /><br />Most sports films revolve around true stories or events, and they often do not work well. But this film hits a 10 on the perfectness scale, even though there were a few minor mistakes here and there.<br /><br />10/10',
  'How do you take a cast of experienced, well-known actors, and put together such a stupid movie? Nimrod Antel has the answer: Armored. Six co-workers at an armored car business decide to steal a large shipment of cash themselves. But, just as they get to first base with their plans, everythi

In [ ]:
train_dataset[:1]

{'review': ['This film was pretty good. I am not too big a fan of baseball, but this is a movie that was made to help understand the meaning of love, determination, heart, etc.<br /><br />Danny Glover, Joseph Gordon-Levitt, Brenda Fricker, Christopher Lloyd, Tony Danza, and Milton Davis Jr. are brought in with a variety of talented actors and understanding of the sport. The plot was believable, and I love the message. William Dear and the guys put together a great movie.<br /><br />Most sports films revolve around true stories or events, and they often do not work well. But this film hits a 10 on the perfectness scale, even though there were a few minor mistakes here and there.<br /><br />10/10'],
 'sentiment': [1],
 '__index_level_0__': [381]}

# Tokenize the Text

Use the BERT tokenizer to preprocess the text.



The batched=True parameter in the map method of Hugging Face Dataset means that the mapping function (in this case, tokenize_function) will process the data in batches (groups of examples) rather than processing one example at a time.

1. padding="max_length"

**Purpose:** Ensures that all tokenized sequences have the same length.

**How It Works:**
If a sequence is shorter than max_length, it will be padded with a special padding token (e.g., [PAD]) until it reaches max_length.

If a sequence is longer than max_length, it won't be padded further.

**Why It’s Important:**
Models like BERT require inputs of uniform length for batch processing.
This ensures that all sequences in a batch can be processed simultaneously.

**2. truncation=True**

**Purpose:** Ensures that sequences longer than max_length are truncated to fit within the limit.

**How It Works:**
If a sequence exceeds the max_length, it will be truncated by removing tokens from the end of the sequence.

**Why It’s Important:**
Prevents exceeding the model's maximum input size, which could cause errors or inefficiency.
Useful for handling datasets with highly variable text lengths.

**3. max_length=128**

**Purpose:** Sets the desired length for input sequences.

**How It Works:**
Determines the fixed size for all tokenized sequences after padding/truncation.

**Why It’s Important:**
Most pre-trained models have a maximum token limit (e.g., 512 for BERT).
Reducing max_length to 128 saves computation time and memory, especially if most sequences are shorter than 128 tokens.

In [ ]:
from transformers import BertTokenizer

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenization function
def tokenize_function(examples):
    return tokenizer(
        examples["review"],
        padding="max_length",
        truncation=True,
        max_length=128
    )
# Tokenize datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)
train_dataset

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/891 [00:00<?, ? examples/s]

Map:   0%|          | 0/223 [00:00<?, ? examples/s]

Dataset({
    features: ['review', 'sentiment', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 891
})

In [ ]:
train_dataset[0].keys()

dict_keys(['review', 'sentiment', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
# Remove unnecessary columns
train_dataset = train_dataset.remove_columns(["review", "__index_level_0__"])
test_dataset = test_dataset.remove_columns(["review", "__index_level_0__"])
train_dataset

Dataset({
    features: ['sentiment', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 891
})

In [ ]:
#train_dataset[0]

In [ ]:
type(train_dataset)

datasets.arrow_dataset.Dataset

**Note**

- transformers models expect output name as **labels**

- here our output column name is **sentiment**

- before convert data to torch type make sure rename the column name from sentiment to labels

In [ ]:
# Set the format for PyTorch
# train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
# test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [ ]:
# Rename the sentiment column to labels
train_dataset = train_dataset.rename_column("sentiment", "labels")
test_dataset = test_dataset.rename_column("sentiment", "labels")

In [ ]:
# Set the format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
train_dataset

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 891
})

In [ ]:
train_dataset[0]

{'labels': tensor(1),
 'input_ids': tensor([  101,  2023,  2143,  2001,  3492,  2204,  1012,  1045,  2572,  2025,
          2205,  2502,  1037,  5470,  1997,  3598,  1010,  2021,  2023,  2003,
          1037,  3185,  2008,  2001,  2081,  2000,  2393,  3305,  1996,  3574,
          1997,  2293,  1010,  9128,  1010,  2540,  1010,  4385,  1012,  1026,
          7987,  1013,  1028,  1026,  7987,  1013,  1028,  6266, 20012,  1010,
          3312,  5146,  1011, 11902,  4779,  1010, 15507, 10424,  6799,  2121,
          1010,  5696,  6746,  1010,  4116,  4907,  4143,  1010,  1998,  9660,
          4482,  3781,  1012,  2024,  2716,  1999,  2007,  1037,  3528,  1997,
         10904,  5889,  1998,  4824,  1997,  1996,  4368,  1012,  1996,  5436,
          2001, 19337,  2666, 12423,  1010,  1998,  1045,  2293,  1996,  4471,
          1012,  2520,  6203,  1998,  1996,  4364,  2404,  2362,  1037,  2307,
          3185,  1012,  1026,  7987,  1013,  1028,  1026,  7987,  1013,  1028,
          2087,  

**so far all together**

In [ ]:
# from transformers import BertTokenizer

# # Initialize the tokenizer
# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# # Tokenization function
# def tokenize_function(examples):
#     return tokenizer(
#         examples["review"],
#         padding="max_length",
#         truncation=True,
#         max_length=128
#     )
# # Tokenize datasets
# train_dataset = train_dataset.map(tokenize_function, batched=True)
# test_dataset = test_dataset.map(tokenize_function, batched=True)

# # Remove unnecessary columns
# train_dataset = train_dataset.remove_columns(["review", "__index_level_0__"])
# test_dataset = test_dataset.remove_columns(["review", "__index_level_0__"])

# # Rename the sentiment column to labels
# train_dataset = train_dataset.rename_column("sentiment", "labels")
# test_dataset = test_dataset.rename_column("sentiment", "labels")

# # Set the format for PyTorch
# train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
# test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])



# Initialize the Model

1.   BertForSequenceClassification
2.   AutoModelForSequenceClassification



Load the BertForSequenceClassification model.

In [ ]:
#!pip install numpy>=1.24.0

In [ ]:
from transformers import BertForSequenceClassification

# Load the model
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2  # Binary classification
)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

# Train the model

https://wandb.ai/omkar-nallagoni-naresh-it

In [ ]:
from transformers import TrainingArguments, Trainer

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results", # where i Need to store results
    eval_strategy="epoch", # evaluation of model per epoch
    learning_rate=2e-5, # learning rate
    per_device_train_batch_size=8, # Per GPU or CPU how many training samples
    per_device_eval_batch_size=8,  # Per GPU or CPU how many evaluation samples
    num_train_epochs=3, # Number of epochs
    weight_decay=0.01,  # regularization
    logging_dir="./logs", # where i need to save the logs
    logging_steps=10,    # after how many steps samples need to logging
    save_steps=1000      # after how many steps we need to save
)
training_args

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=IntervalStrategy.EPOCH,
eval_use_gather_object=False

**Trainer**

- The trainer requires

    - Model

    - arguments

    - Train datasets

    - Eval dataset

In [ ]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer


In [ ]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shivraj066621 (shivraj066621-nareshit) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.371300,0.317420
2,0.139600,0.372479
3,0.105600,0.407107


TrainOutput(global_step=336, training_loss=0.2797225107926698, metrics={'train_runtime': 255.4473, 'train_samples_per_second': 10.464, 'train_steps_per_second': 1.315, 'total_flos': 175823962744320.0, 'train_loss': 0.2797225107926698, 'epoch': 3.0})

In [ ]:
results = trainer.evaluate()
print(results)


{'eval_loss': 0.40710681676864624, 'eval_runtime': 1.5356, 'eval_samples_per_second': 145.221, 'eval_steps_per_second': 18.234, 'epoch': 3.0}


# Save the model

In [ ]:
# model is a varaible used at the time of reading the model
# tokenizer is a variable used at the time of reading tokenizer model
# tokenzier use to generate
#       token ids , attention mask
# model will generate the embeddings and train word embeddings
#      token ids and attention mask
#      model will internally generate the emeddings
#     training
model.save_pretrained("./sentiment_model")
tokenizer.save_pretrained("./sentiment_model")


('./sentiment_model/tokenizer_config.json',
 './sentiment_model/special_tokens_map.json',
 './sentiment_model/vocab.txt',
 './sentiment_model/added_tokens.json')

# Load the saved model

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load the model and tokenizer
model = BertForSequenceClassification.from_pretrained("./sentiment_model")
tokenizer = BertTokenizer.from_pretrained("./sentiment_model")


# Predict on unseen data

In [ ]:
review = "I really not like this movie but The story was good!"
inputs = tokenizer(
        review,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=256
    )

print(inputs)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device
model.to(device)
inputs = {key: value.to(device) for key, value in inputs.items()}
inputs

with torch.no_grad():
    outputs = model(**inputs)
    print(outputs)

import numpy as np
torch.argmax(outputs.logits)

{'input_ids': tensor([[ 101, 1045, 2428, 2025, 2066, 2023, 3185, 2021, 1996, 2466, 2001, 2204,
          999,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
SequenceClassifierOutput(loss=None, logits=tensor([[-2.0880,  2.0014]], device='cuda:0'), hidden_states=None, attentions=None)


tensor(1, device='cuda:0')

**logits and probabilites**

- logits are raw values

- logits are any real values that might be postive and negative

- logits will pass through activation functions either sigmoid or softmax

- those activation functions will give the probabilties

In [ ]:
review = "I really not like this movie"
inputs = tokenizer(
        review,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=256
    )

print(inputs)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device
model.to(device)
inputs = {key: value.to(device) for key, value in inputs.items()}
inputs

with torch.no_grad():
    outputs = model(**inputs)
    print(outputs)

import numpy as np
torch.argmax(outputs.logits)

{'input_ids': tensor([[ 101, 1045, 2428, 2025, 2066, 2023, 3185,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}
SequenceClassifierOutput(loss=None, logits=tensor([[ 1.5198, -1.4150]], device='cuda:0'), hidden_states=None, attentions=None)


tensor(0, device='cuda:0')

In [ ]:
def predict_sentiment(review, model, tokenizer):
    # Tokenize the input review
    inputs = tokenizer(
        review,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=256
    )

    # Move tensors to the same device as the model (CPU or GPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Perform prediction
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

    # Map predictions to sentiment labels
    sentiment = "positive" if predictions.item() == 1 else "negative"
    return sentiment


In [ ]:
review = "I really loved this movie. The story was  good!"
sentiment = predict_sentiment(review, model, tokenizer)\

print(f"Review: {review}")
print(f"Predicted Sentiment: {sentiment}")


Review: I really loved this movie. The story was  good!
Predicted Sentiment: positive


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)
trainer.train()


TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'